In [ ]:
import binaryninja as bn
import monkeyhex, ctypes

In [ ]:
'''
#include <cstdint>
#include <vector>
#include <random>
#include <algorithm>

extern "C"{
void rng(uint8_t* buf, int len, uint8_t seed){
    std::vector<uint8_t> ubw;
    std::mt19937 g(seed);
    for(int i = 0; i < len; i++){
        ubw.push_back(i);
    }
    std::shuffle(ubw.begin(), ubw.end(), g);
    for(int i = 0; i < len; i++){
        buf[i] = ubw[i];
    }
}
}
'''
lrng=ctypes.CDLL('./librng.so')
rng = lrng.rng
rng.argtypes = [ctypes.c_char_p, ctypes.c_int, ctypes.c_ubyte]

In [ ]:
buff = ctypes.create_string_buffer(int(192))

In [ ]:
# orig is compiled without applying watermark
orig=bn.BinaryViewType['ELF'].open('orig')
orig.update_analysis_and_wait()

In [ ]:
lorig=list(orig.functions)
lforig=[lorig[22],lorig[24],lorig[26],lorig[28],lorig[29],lorig[30]]

In [ ]:
def bb2depth(head):
    depth = 0
    currb = [head]
    bbdepth = {}

    while currb:
        # print(depth, currb)
        nextb = []
        for bb in currb:
            if bb.instruction_count == 1:
                bb = bb.outgoing_edges[0].target
            if bb.start not in bbdepth:
                bbdepth[bb.start] = [depth]
            else:
                if depth not in bbdepth[bb.start]:
                    bbdepth[bb.start].append(depth)
                continue
            for e in bb.outgoing_edges:
                nextb.append(e.target)
        depth += 1
        currb = nextb
    return bbdepth

def cantor(order):
    result = 0
    for i in range(1,len(order)-1):
        c = 0
        for j in range(i,len(order)):
            if order[j] < order[i]:
                c += 1
        result += c
        result *= len(order) - i - 1
    return result
cantor_len = [0,  0,  1,  2,  4,  6,  9,  12, 15, 18, 21, 25,  28,  32,  36,  40,  44, 48, 52, 56, 61, 65, 69, 74, 79, 83, 88, 93, 97, 102, 107, 112, 117, 122, 127]

In [ ]:
 b64idx = [
    255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
    255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
    255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
    255, 255, 255, 57,  20,  255, 4,   11,  45,  38,  43,  56,  39,  255, 255,
    255, 255, 255, 255, 255, 6,   36,  41,  63,  12,  21,  58,  10,  40,  33,
    59,  54,  15,  3,   31,  5,   47,  25,  61,  2,   26,  34,  9,   53,  18,
    1,   255, 255, 255, 255, 29,  255, 55,  0,   50,  8,   46,  17,  19,  35,
    7,   62,  52,  16,  60,  32,  44,  13,  22,  42,  51,  37,  30,  24,  48,
    23,  14,  27,  49,  255, 28,  255, 255, 255, 255, 255, 255, 255, 255, 255,
    255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
    255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
    255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
    255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
    255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
    255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
    255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
    255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
    255]

In [ ]:
answers = open('binaries/results.txt').read().splitlines()

In [ ]:
secret = []
for elf in range(1, 257):
    print(elf)
    bv=bn.BinaryViewType['ELF'].open(f'{elf}')
    bv.update_analysis_and_wait()

    l=list(bv.functions)
    lf=[l[22],l[24],l[26],l[28],l[29],l[30]]

    bindata = ''
    remaining = 192
    for findex in range(6):
        cforig=lforig[findex]
        cfborig=sorted(list(filter(lambda bb:bb.instruction_count > 1, cforig.basic_blocks)), key=lambda bb:bb.start)[:35]
        cf=lf[findex]
        cfb=sorted(list(filter(lambda bb:bb.instruction_count > 1, cf.basic_blocks)), key=lambda bb:bb.start)[:35]

        bdepthorig = bb2depth(cfborig[0])
        bdepth = bb2depth(cfb[0])

        insdict = {'jmp': 1}
        currP = 2
        for ins in cforig.instructions:
            opcode = str(ins[0][0])
            if opcode not in insdict:
                insdict[opcode] = currP
                currP = next_prime(currP)
        for bbstart in bdepthorig:
            sd = str(bdepthorig[bbstart])
            if sd not in insdict:
                insdict[sd] = currP
                currP = next_prime(currP)
        for ins in cf.instructions:
            opcode = str(ins[0][0])
            if opcode not in insdict:
                insdict[opcode] = currP
                currP = next_prime(currP)

        bbIsorig = [reduce(lambda a,b:a*b, [insdict[str(ins[0][0])] for ins in bb])*insdict[str(bdepthorig[bb.start])] for bb in cfborig]
        bbIs = [reduce(lambda a,b:a*b, [insdict[str(ins[0][0])] for ins in bb])*insdict[str(bdepth[bb.start])] for bb in cfb]

        gorder = Matrix(QQ,len(bbIsorig),len(bbIsorig))
        for i in range(len(bbIsorig)):
            g = list(map(lambda x: abs(1-2*reduce(lambda a,b: (0,a[1]+b[1]),factor(2*gcd(bbIs[i], x)))[1]/reduce(lambda a,b: (0,a[1]+b[1]),factor(4*x*bbIs[i]))[1]), bbIsorig))
            gorder[i] = vector(g)

        order = [-1]*len(bbIsorig)
        for _ in range(len(bbIsorig)):
            m = list(map(min,gorder))
            i = m.index(min(m))
            j = list(gorder[i]).index(min(m))
            order[i] = j
            gorder[i,:] = 100
            gorder[:,j] = 100
        data = format(cantor(order), f'0{cantor_len[len(order)-1]}b')

        if findex == 0:
            tlen = int(data[1:6],2)*6+6
            remaining = tlen
        bindata += data[::-1][:remaining][::-1]
        remaining -= cantor_len[len(order)-1]
        if remaining <=0 :
            break

    rng(buff,ctypes.c_int(tlen-6),ctypes.c_ubyte(int(bindata[:6],2)))
    order=list(map(int, buff.raw))[:tlen-6]
    obdata = ''.join([bindata[6+order.index(i)] for i in range(tlen-6)])
    secret = ''.join([chr(b64idx.index(int(obdata[i*6:i*6+6], 2))) for i in range((tlen-6)//6)])
    print(secret == answers[elf-1])